In [1]:
import os
import pandas as pd
from pathlib import Path

In [ ]:
blast_file_dir = Path("/example/path/to/blast/output/")
ref_path = Path("/example/path/to/refseq/")

genes = [os.path.basename(file).split("_query")[0] for file in os.listdir(blast_file_dir) if file.endswith(".out")]
print(genes)

In [24]:
def find_off_targets(gene, blast_query_path, ref_path, armlength=20):
    print(f"Finding off-targets for {gene}", flush=True)
    file = blast_query_path / f"{gene}_query_blast.out"

    df = pd.read_csv(file, header=None)

    # Exclude lines with predicted transcripts
    df = df.loc[~(df[1].str.contains("XR") | df[1].str.contains("XM"))]

    # Exclude hits with less than 50% coverage and less than 80% homology
    #df = df.loc[df[2] > 80]
    #df = df[df[3] > 2*armlength*0.5]

    # Penalise gaps in the hits
    #df = df.loc[df[6] < armlength - 4]
    #df = df.loc[df[7] > armlength + 5]

    variants = find_variants(gene, ref_path)

    # Exclude known variants 
    df = df.loc[~(df[1].isin(variants))]

    df['gene'] = gene

    # Write off-targets to file 
    df.to_csv(blast_query_path / f"{gene}_off_targets.out")


def find_variants(gene, ref_path):
    """This function finds variants from the .acronymheaders.txt, selectedseqs.text and selectedheaders.txt files made by multi_padlock_design scripts"""
    acronyms = pd.read_csv(ref_path / "mouse.acronymheaders.txt", header=None)
    acronyms.columns = ["gene_name"]

    indices = acronyms.index[acronyms.gene_name == gene].to_list()

    accession_numbers = []
    with open(ref_path / "mouse.selectedheaders.txt", "r") as f:
        lines = f.readlines()
        for i in indices:
            accession_numbers.append(lines[int(i)].strip())
    
    accession_numbers = format_variants(accession_numbers)

    return accession_numbers

def format_variants(accession_numbers):
    # Remove the > character and take everything before the first whitespace character
    accession_numbers = [accession.replace(">", "") for accession in accession_numbers]
    accession_numbers = [accession.split(" ")[0] for accession in accession_numbers]
    return accession_numbers 


In [ ]:
for gene in genes:
    find_off_targets(gene, blast_file_dir, ref_path)

In [3]:
#make a df from all the files that end with _off_targets.out
import natsort

import glob
import pandas as pd
files = glob.glob(blast_file_dir / "*_off_targets.out")
# ignore the first row of each file
df = pd.concat([pd.read_csv(file, header=0) for file in files])
# sort df with natsort on column
df = df.iloc[natsort.index_natsorted(df["0"])]
header = ["query", "subject", "percentage identity", "length", "mismatches", "gaps", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "sseq", "gene"]
#remove the first column
df = df.drop(columns=["Unnamed: 0"])
df.columns = header
df.sort_values(by=["evalue"], inplace=True)
df_filtered = df[df["evalue"] < 1]
df_filtered

In [ ]:
import os
from collections import defaultdict
import pandas as pd
import config
from tqdm import tqdm

# Cached data
cached_data = {}

def loaddb(species, config):
    """ Load formatted RefSeq header and sequence files """
    fastadir = (config.fastadir_mouse, config.fastadir_human)
    fasta_filenum = (config.fasta_filenum_mouse, config.fasta_filenum_human)
    fasta_pre_suffix = (config.fasta_pre_suffix_mouse, config.fasta_pre_suffix_human)

    if species == "mouse":
        s = 0
    elif species == "human":
        s = 1
    else:
        raise ValueError(f"Unknown species: {species}")

    if species not in cached_data:
        if not os.path.isfile(fastadir[s] + '/' + species + '.acronymheaders.txt'):
            print("Processing fasta database files..")
            formatrefseq.fastadb(fastadir[s], fasta_filenum[s], fasta_pre_suffix[s], species)
        
        with open(fastadir[s] + '/' + species + '.acronymheaders.txt', 'r') as f:
            Acronyms = [line.rstrip('\n') for line in f]
        with open(fastadir[s] + '/' + species + '.selectedheaders.txt', 'r') as f:
            Headers = [line.rstrip('\n') for line in f]
        with open(fastadir[s] + '/' + species + '.selectedseqs.txt', 'r') as f:
            Seq = [line.rstrip('\n') for line in f]
        
        cached_data[species] = (Acronyms, Headers, Seq)
    return cached_data[species]

def find_genes_from_variants(variants, species, config):
    """ Find genes (acronyms) from a list of variants """
    Acronyms, Headers, Seq = loaddb(species, config)
    
    variant_to_acronym = defaultdict(list)
    for acronym, header in zip(Acronyms, Headers):
        gene_variant = header[1:].split('.', 1)[0]
        variant_to_acronym[gene_variant].append(acronym)

    genes = [variant_to_acronym[variant] if variant in variant_to_acronym else None for variant in variants]
    
    return genes

# Assuming `config` is an object with the necessary attributes
loaddb("mouse", config)

# Optimized DataFrame processing
def get_genes(subject):
    variant = subject.split('.', 1)[0]
    return find_genes_from_variants([variant], 'mouse', config)

tqdm.pandas()
df_filtered['offtarget'] = df_filtered['subject'].progress_map(get_genes)
df_filtered['offtarget']= df_filtered['offtarget'].apply(lambda x: x[0][0] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], list) and len(x[0]) > 0 else x)

In [ ]:
import matplotlib.pyplot as plt
#plot sorted evalues
plt.figure(dpi=300)
# Define the window size for the rolling mean
window_size = 1  # Adjust this value based on your data

df_sorted_by_evalue = df_filtered.sort_values(by='evalue')

# Calculate the rolling mean of 'length'
df_sorted_by_evalue['length_smooth'] = df_sorted_by_evalue['length'].rolling(window=window_size).mean()

# Create a figure and a single subplot
fig, ax1 = plt.subplots(dpi=300)

# Plot evalue on the first y-axis
ax1.plot(df_sorted_by_evalue["evalue"].reset_index(drop=True), color='blue')
ax1.set_yscale("log")
ax1.set_xlabel("Off-target hits")
ax1.set_ylabel("E-value", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot smoothed length on the second y-axis
ax2.plot(df_sorted_by_evalue["length_smooth"].reset_index(drop=True), color='red', alpha=0.5)
ax2.set_ylabel("Length", color='red')  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor='red')
plt.xlim(0, 1000)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# Calculate the rolling mean of 'percentage identity'
df_sorted_by_evalue['percentage_identity_smooth'] = df_sorted_by_evalue['percentage identity'].rolling(window=window_size).mean()

# Create a figure and a single subplot
fig, ax1 = plt.subplots(dpi=300)

# Plot evalue on the first y-axis
ax1.plot(df_sorted_by_evalue["evalue"].reset_index(drop=True), color='blue')
ax1.set_yscale("log")
ax1.set_xlabel("Off-target hits")
ax1.set_ylabel("E-value", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot smoothed percentage identity on the second y-axis
ax2.plot(df_sorted_by_evalue["percentage_identity_smooth"].reset_index(drop=True), color='green', alpha=0.5)
ax2.set_ylabel("Percentage Identity", color='green')  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor='green')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# Calculate the sum of 'mismatches' and 'gaps', then calculate the rolling mean
df_sorted_by_evalue['mismatches_gaps'] = df_sorted_by_evalue['mismatches'] + df_sorted_by_evalue['gaps']
df_sorted_by_evalue['mismatches_gaps_smooth'] = df_sorted_by_evalue['mismatches_gaps'].rolling(window=window_size).mean()

# Create a figure and a single subplot
fig, ax1 = plt.subplots(dpi=300)

# Plot evalue on the first y-axis
ax1.plot(df_sorted_by_evalue["evalue"].reset_index(drop=True), color='blue')
ax1.set_yscale("log")
ax1.set_xlabel("Off-target hits")
ax1.set_ylabel("E-value", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot smoothed mismatches+gaps on the second y-axis
ax2.plot(df_sorted_by_evalue["mismatches_gaps_smooth"].reset_index(drop=True), color='orange', alpha=0.5)
ax2.set_ylabel("Mismatches+Gaps", color='orange')  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor='orange')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# Assuming df_filtered is already defined

# Initialize the DataFrame
df_padlock = pd.DataFrame(columns=["padlock_name", "gene", "offtarget"])

# Get unique genes
unique_padlocks = df_filtered["query"].unique()

# Helper function to strip the extra list brackets from the strings
def strip_extra_brackets(offtarget_list):
    return [item.strip("['']") for item in offtarget_list]

# Populate the DataFrame
data = []
for padlock in tqdm(unique_padlocks):
    gene = df_filtered[df_filtered["query"] == padlock]["gene"].iloc[0]
    offtargets = df_filtered[df_filtered["query"] == padlock]["offtarget"].tolist()
    
    # Strip the extra brackets
    #stripped_offtargets = strip_extra_brackets(offtargets)
    
    # Get unique offtargets
    unique_offtargets = np.unique(offtargets)
    
    data.append({"padlock_name": padlock, "gene": gene, "offtarget": unique_offtargets.tolist()})

df_padlock = pd.DataFrame(data)
df_padlock

# Group by gene and aggregate off-targets
df_gene = df_padlock.groupby("gene")["offtarget"].apply(lambda x: np.unique(np.concatenate(x.tolist()))).reset_index()
df_gene

In [ ]:
import mygene
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Initialize the MyGene client
mg = mygene.MyGeneInfo()

# Function to get the main symbols for a gene (up to 5 hits)
def get_main_symbols(gene):
    retries = 5
    backoff_factor = 0.5
    for i in range(retries):
        try:
            result = mg.query(gene, species='mouse', size=20)  # Change to 'human' if using human genes
            if 'hits' in result and result['hits']:
                return [hit.get('symbol', 'No symbol found') for hit in result['hits']]
            return ['No symbol found']
        except HTTPError as e:
            if e.response.status_code == 429:
                wait = backoff_factor * (2 ** i)  # Exponential backoff
                print(f"Rate limited. Retrying in {wait} seconds...")
                time.sleep(wait)
            else:
                raise e
    return ['No symbol found']

# Function to process each row
def process_row(row):
    gene = row.gene
    offtarget_symbols = row.offtarget
    
    # Get the main symbols for the gene
    main_symbols = get_main_symbols(gene)
    
    # Filter out the main symbols from the offtarget list
    filtered_symbols = [symbol for symbol in offtarget_symbols if symbol not in main_symbols]
    
    if filtered_symbols:  # Only add rows where the filtered list is not empty
        return filtered_symbols
    else:
        return None

# Perform the main symbol queries in parallel
with ThreadPoolExecutor() as executor:
    filtered_offtarget = list(tqdm(executor.map(process_row, df_padlock.itertuples(index=False)), total=len(df_padlock)))

# Create a new DataFrame with the filtered offtarget lists
df_padlock_filtered = df_padlock.copy()
df_padlock_filtered['offtarget'] = filtered_offtarget

# Remove rows where offtarget is None (i.e., all genes were deleted)
df_padlock_filtered = df_padlock_filtered[df_padlock_filtered['offtarget'].notna()]

df_padlock_filtered.reset_index(drop=True, inplace=True)


In [ ]:
import mygene
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Initialize the MyGene client
mg = mygene.MyGeneInfo()

# Function to get the main symbols for a gene (up to 5 hits)
def get_main_symbols(gene):
    result = mg.query(gene, species='mouse', size=20)
    if 'hits' in result and result['hits']:
        return [hit.get('symbol', 'No symbol found') for hit in result['hits']]
    return ['No symbol found']

# Function to process each row
def process_row(row):
    gene = row.gene
    offtarget_symbols = row.offtarget
    
    # Get the main symbols for the gene
    main_symbols = get_main_symbols(gene)
    
    # Filter out the main symbols from the offtarget list
    filtered_symbols = [symbol for symbol in offtarget_symbols if symbol not in main_symbols]
    
    if filtered_symbols:  # Only add rows where the filtered list is not empty
        return filtered_symbols
    else:
        return None

# Perform the main symbol queries in parallel
with ThreadPoolExecutor() as executor:
    filtered_offtarget = list(tqdm(executor.map(process_row, df_gene.itertuples(index=False)), total=len(df_gene)))

# Create a new DataFrame with the filtered offtarget lists
df_gene_filtered = df_gene.copy()
df_gene_filtered['offtarget'] = filtered_offtarget

# Remove rows where offtarget is None (i.e., all genes were deleted)
df_gene_filtered = df_gene_filtered[df_gene_filtered['offtarget'].notna()]

df_gene_filtered.reset_index(drop=True, inplace=True)

In [22]:
df_padlock_filtered.to_csv("/example_path/df_padlock_filtered.csv", index=False)
df_gene_filtered.to_csv("/example_path/df_gene_filtered.csv", index=False)

In [ ]:
all_probes = pd.read_csv("/example/path/to/original_padlocks.csv")

In [ ]:
import matplotlib.pyplot as plt
# Calculating the number of padlocks per gene from all_probes
padlocks_per_gene_all_probes = all_probes['gene_name'].value_counts()

# Calculating the number of padlocks per gene from df_padlock
padlocks_per_gene_df_padlock = df_padlock_filtered['gene'].value_counts()

# Combining the two counts into one DataFrame for plotting
genes = padlocks_per_gene_all_probes.index.union(padlocks_per_gene_df_padlock.index)
combined_counts = pd.DataFrame({
    'Total Padlocks': padlocks_per_gene_all_probes.reindex(genes, fill_value=0),
    'Padlocks in df_padlock': padlocks_per_gene_df_padlock.reindex(genes, fill_value=0)
})

# Plotting the data
fig, ax = plt.subplots(figsize=(60, 6))
combined_counts['Total Padlocks'].plot(kind='bar', color='black', ax=ax, position=0, width=0.4)
combined_counts['Padlocks in df_padlock'].plot(kind='bar', color='red', ax=ax, position=1, width=0.4)

# Customizing the plot
ax.set_title('Off target padlocks per Gene')
ax.set_xlabel('Gene')
ax.set_ylabel('Number of Padlocks')
ax.legend(['Total Padlocks', 'Off-target Padlocks'])
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()